# Catalogue Data to Linked Art - Indianapolis Museum of Art

## Introduction

Linked Art is a community working together to create a shared Model based on Linked Open Data to describe Art. 

- This Jupyter notebook provides a documented exemplar of the transformation process, from catalogue data to Linked Art.

- Further information: https://linked.art

In [116]:
from IPython.display import IFrame
IFrame("https://linked.art", width=1000, height=350)

## Indianapolis Museum of Art (IMA) & Linked Art
The [Indianapolis Museum of Art (IMA)](https://discovernewfields.org/) has transformed a sample of its catalogue data to Linked Art JSON-LD. 

- The data are available at https://github.com/IMAmuseum/LinkedArt 
- Data were sourced from EMu in XML format (EMu source: Catalogue, Rights, Narratives, and Locations modules)
- This Jupyter notebook uses the [Objects catalogue data](https://github.com/IMAmuseum/LinkedArt/blob/master/XML/ObjectsSample.xml) to provided a documented exemplar of the transformation process, from catalogue data to Linked Art
- [raw file](https://raw.githubusercontent.com/IMAmuseum/LinkedArt/master/XML/ObjectsSample.xml) 

In [117]:
from IPython.display import IFrame
IFrame('https://discovernewfields.org/collections', width='100%', height=600)

## Catalogue Data 
### Import What We Need for Notebook
- Import Python libraries
- Add external javascript libraries for JSON-LD and d3 as well as style definitions
- Add third-party code for JSON-LD graph visualisation

In [118]:
try:
    import ipywidgets as widgets
except:
    !pip install ipywidgets
    import ipywidgets as widgets

try:
    import IPython
except:
    !pip install IPython
    import IPython   
    
    
try:
    import xmltodict
except:
    !pip install xmltodict
    import xmltodict

try:
    import json
except:
    !pip install json
    import json  
        
from ipywidgets import Layout

def widgeText(desc, jdoc, ht):
    widg = widgets.Textarea(
        value=json.dumps(jdoc, indent=2),
        placeholder="",
        description=desc,
        disabled=False,
        layout=Layout(width='100%', height=ht))
    return widg

In [119]:
%%html
<script src="https://cdn.jsdelivr.net/npm/jsonld@3.2.0/dist/jsonld.min.js"></script> 
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.17/d3.min.js"></script>  

<style>

.widget-label{
    width:150px;
}
.output_wrapper{
    height:auto;
}
#visualized{
    width:2000px !important;
    height:1000px;
}

div.svg{
        width:4000px !important;
        height:4000px;
}

div.svg-small{
    width:2000px !important;
    height:500px;
}

div.output_area{
    background-color:whitesmoke;
}

@import url("https://fonts.googleapis.com/css?family=Open+Sans:300,400,600");

svg {
  border: none;
}

.node {
  cursor: pointer;
}

.node text {
  font-size: 12px;
  font-family: 'Open Sans', 'Helvetica Neue', Helvetica, sans-serif;
  fill: #333333;
}

.d3-tip {
  font-size: 14px;
  font-family: 'Open Sans', 'Helvetica Neue', Helvetica, sans-serif;
  color: #333333;
  border: 1px solid #CCCCCC;
  border-radius: 5px;
  padding: 10px 20px;
  max-width: 250px;
  word-wrap: break-word;
  background-color: rgba(255, 255, 255, 0.9);
  text-align: left;
}

.link {
  fill: none;
  stroke: #DADFE1;
  stroke-width: 1px;
}

</style>



In [121]:
%%javascript
/* modified from https://json-ld.org/playground/jsonld-vis.js */
  
function jsonldVis(jsonld, selector, config) {
    if (!arguments.length) return jsonldVis;
    config = config || {};

    var h = config.h || 600
      , w = config.w || 800
      , maxLabelWidth = config.maxLabelWidth || 250
      , transitionDuration = config.transitionDuration || 750
      , transitionEase = config.transitionEase || 'cubic-in-out'
      , minRadius = config.minRadius || 5
      , scalingFactor = config.scalingFactor || 2;

    var i = 0;

    var tree = d3.layout.tree()
      .size([h, w]);

    var diagonal = d3.svg.diagonal()
      .projection(function(d) { return [d.y, d.x]; });
      
    
    var svg = d3.select(selector).append('svg')
      .attr('width', w)
      .attr('id', selector + "-svg")
      .attr('xmlns', 'http://www.w3.org/2000/svg')
      .attr('height', h)
      .append('g')
      .attr('transform', 'translate(' + maxLabelWidth + ',0)')
      

    var root = jsonldTree(jsonld);
    root.x0 = h / 2;
    root.y0 = 0;
    root.children.forEach(collapse);

    function changeSVGWidth(newWidth) {
      if (w !== newWidth) {
        d3.select(selector + ' > svg').attr('width', newWidth);
      }
    }

    function jsonldTree(source) {
      var tree = {};

      if ('@id' in source) {
        tree.isIdNode = true;
        tree.name = source['@id'];
        if (tree.name.length > maxLabelWidth / 9) {
          tree.valueExtended = tree.name;
          tree.name = '...' + tree.valueExtended.slice(-Math.floor(maxLabelWidth / 9));
        }
      } else {
        tree.isIdNode = true;
        tree.isBlankNode = true;
        // random id, can replace with actual uuid generator if needed
        tree.name = '_' + Math.random().toString(10).slice(-7);
      }

      var children = [];
      Object.keys(source).forEach(function(key) {
        if (key === '@id' || key === '@context' || source[key] === null) return;

        var valueExtended, value;
        if (typeof source[key] === 'object' && !Array.isArray(source[key])) {
          children.push({
            name: key,
            children: [jsonldTree(source[key])]
          });
        } else if (Array.isArray(source[key])) {
          children.push({
            name: key,
            children: source[key].map(function(item) {
              if (typeof item === 'object') {
                return jsonldTree(item);
              } else {
                return { name: item };
              }
            })
          });
        } else {
          valueExtended = source[key];
          value = valueExtended;
          if (value.length > maxLabelWidth / 9) {
            value = value.slice(0, Math.floor(maxLabelWidth / 9)) + '...';
            children.push({
              name: key,
              value: value,
              valueExtended: valueExtended
            });
          } else {
            children.push({
              name: key,
              value: value
            });
          }
        }
      });

      if (children.length) {
        tree.children = children;
      }

      return tree;
    }

    function update(source) {
      var nodes = tree.nodes(root).reverse();
      var links = tree.links(nodes);

      nodes.forEach(function(d) { d.y = d.depth * maxLabelWidth; });

      var node = svg.selectAll('g.node')
        .data(nodes, function(d) { return d.id || (d.id = ++i); });

      var nodeEnter = node.enter()
        .append('g')
        .attr('class', 'node')
        .attr('transform', function(d) { return 'translate(' + source.y0 + ',' + source.x0 + ')'; })
        .on('click', click);

      nodeEnter.append('circle')
        .attr('r', 0)
        .style('stroke-width', function(d) {
          return d.isIdNode ? '2px' : '1px';
        })
        .style('stroke', function(d) {
          return d.isIdNode ? '#F7CA18' : '#4ECDC4';
        })
        .style('fill', function(d) {
          if (d.isIdNode) {
            return d._children ? '#F5D76E' : 'white';
          } else {
            return d._children ? '#86E2D5' : 'white';
          }
        })
    
    nodeEnter.append('title')
        .text(function(d) { return  (d.valueExtended) ?  d.valueExtended : "" })
        
      nodeEnter.append('text')
        .attr('x', function(d) {
          var spacing = computeRadius(d) + 5;
          return d.children || d._children ? -spacing : spacing;
        })
        .attr('dy', '4')
        .attr('text-anchor', function(d) { return d.children || d._children ? 'end' : 'start'; })
        .text(function(d) { return d.name + (d.value ? ': ' + d.value : ''); })
        .style('fill-opacity', 0);

      var maxSpan = Math.max.apply(Math, nodes.map(function(d) { return d.y + maxLabelWidth; }));
      if (maxSpan + maxLabelWidth + 20 > w) {
        changeSVGWidth(maxSpan + maxLabelWidth);
        d3.select(selector).node().scrollLeft = source.y0;
      }

      var nodeUpdate = node.transition()
        .duration(transitionDuration)
        .ease(transitionEase)
        .attr('transform', function(d) { return 'translate(' + d.y + ',' + d.x + ')'; });

      nodeUpdate.select('circle')
        .attr('r', function(d) { return computeRadius(d); })
        .style('stroke-width', function(d) {
          return d.isIdNode ? '2px' : '1px';
        })
        .style('stroke', function(d) {
          return d.isIdNode ? '#F7CA18' : '#4ECDC4';
        })
        .style('fill', function(d) {
          if (d.isIdNode) {
            return d._children ? '#F5D76E' : 'white';
          } else {
            return d._children ? '#86E2D5' : 'white';
          }
        });

      nodeUpdate.select('text').style('fill-opacity', 1);

      var nodeExit = node.exit().transition()
        .duration(transitionDuration)
        .ease(transitionEase)
        .attr('transform', function(d) { return 'translate(' + source.y + ',' + source.x + ')'; })
        .remove();

      nodeExit.select('circle').attr('r', 0);
      nodeExit.select('text').style('fill-opacity', 0);

      var link = svg.selectAll('path.link')
        .data(links, function(d) { return d.target.id; });

      link.enter().insert('path', 'g')
        .attr('class', 'link')
        .attr('d', function(d) {
          var o = { x: source.x0, y: source.y0 };
          return diagonal({ source: o, target: o });
        });

      link.transition()
        .duration(transitionDuration)
        .ease(transitionEase)
        .attr('d', diagonal);

      link.exit().transition()
        .duration(transitionDuration)
        .ease(transitionEase)
        .attr('d', function(d) {
          var o = { x: source.x, y: source.y };
          return diagonal({ source: o, target: o });
        })
        .remove();

      nodes.forEach(function(d) {
        d.x0 = d.x;
        d.y0 = d.y;
      });
    }

    function computeRadius(d) {
      if (d.children || d._children) {
        return minRadius + (numEndNodes(d) / scalingFactor);
      } else {
        return minRadius;
      }
    }

    function numEndNodes(n) {
      var num = 0;
      if (n.children) {
        n.children.forEach(function(c) {
          num += numEndNodes(c);
        });
      } else if (n._children) {
        n._children.forEach(function(c) {
          num += numEndNodes(c);
        });
      } else {
        num++;
      }
      return num;
    }

    function click(d) {
      if (d.children) {
        d._children = d.children;
        d.children = null;
      } else {
        d.children = d._children;
        d._children = null;
      }

      update(d);

      // fast-forward blank nodes
      if (d.children) {
        d.children.forEach(function(child) {
          if (child.isBlankNode && child._children) {
            click(child);
          }
        });
      } }

    function collapse(d) {
      if (d.children) {
        d._children = d.children;
        d._children.forEach(collapse);
        d.children = null;
      } }

    update(root);
  }

  if (typeof module !== 'undefined' && module.exports) {
    module.exports = jsonldVis;
  } else {
    d3.jsonldVis = jsonldVis;
  }

jsonldVis(d3);



<IPython.core.display.Javascript object>

In [138]:
%%javascript 

function jsonvis(filename, div){
    d3.json(filename, (err, data) => {
      if (err) return console.warn(err);
      d3.jsonldVis(data, div , { w: 1000, h: 300, maxLabelWidth: 250 });});
    }

<IPython.core.display.Javascript object>

## Data Transformation 
### Define Variables
Please enter a baseURI - example value for IMA is `https://data.discovernewfields.org/`

In [122]:
baseURI = input()

https://abc.org/


### Upload XML File
- Choose a file on your local system to upload and transform to Linked Art
- The IMA files are available to download from: https://github.com/IMAmuseum/LinkedArt/tree/master/XML

In [123]:
from ipywidgets import FileUpload
from IPython.display import display

upload = FileUpload(accept='.xml', multiple=False, description='Select XML file')

In [125]:
display(upload)

obj = "UNDEFINED"

# get content from uploaded file 
for uploaded_filename in upload.value:
    content = upload.value[uploaded_filename]['content']
    obj = xmltodict.parse(content) 
    
if obj == "UNDEFINED":
    print("Please select a file to transform")

FileUpload(value={'ObjectsSample.xml': {'metadata': {'name': 'ObjectsSample.xml', 'type': 'text/xml', 'size': …

In [126]:
allObjects = obj["table"]["tuple"]

#object dropdown options
objOptions = []
objOptions.append(("Please select an object",''))

for obj in allObjects:
    title = irn = ""
    # define properties variables  
    for prop in obj["atom"]:      
        propName = prop["@name"]     
        if "#text" in prop:
            if propName == "irn":
                irn = prop["#text"]  
            if propName == "TitMainTitle":
                title = prop["#text"]
            
    objOptions.append((irn + ' : ' + title,irn))
    
selectObject = widgets.Dropdown(
        options=objOptions,
        description='Select Object')

### Select Catalogue Item to Transform

In [127]:
from IPython.display import display
display(selectObject)

Dropdown(description='Select Object', options=(('Please select an object', ''), ('1032 : long-neck vase with c…

In [128]:
selectedIRN = selectObject.value

selectedObject = {}

# display selected record 
for obj in allObjects:
    for prop in obj["atom"]:         
        if prop["@name"] == "irn":
            irn = prop["#text"]
            if irn == selectedIRN:
                selectedObject = obj
               
widgeText("Artwork", selectedObject,'200px' )

Textarea(value='{\n  "atom": [\n    {\n      "@name": "irn",\n      "@type": "text",\n      "@size": "short",\…

### Minimum Linked Art representation

<a id='core_properties'></a>

#### Core Properties
https://linked.art/model/base/#core-properties

There are a few core properties that every resource should have for it to be a useful part of the world of Linked Open Data:

- @context
- id
- type
- _label
The simplest possible object has a URI, a class and a label.

##### Mapping
- The `id` is a URL and has been created from the `irn` value together with a URL prefix: https://data.discovernewfields.org/

- The `_label` is a human readable label, intended for developers and other people reading the data. The value is taken from the `TitMainTitle` property.

In [129]:
# create dictionary to hold object desription
minla = {}
obj = selectedObject

# define properties variables  
for prop in obj["atom"]:      
    propName = prop["@name"]     
    if "#text" in prop:
        if propName == "irn":
            irn = prop["#text"]  
        if propName == "TitMainTitle":
            title = prop["#text"]

# minimum Linked Art properties
minla["@context"] = "https://linked.art/ns/v1/linked-art.json"
minla["id"] = baseURI + "object/" + irn
minla["type"] = "HumanMadeObject"
minla["_label"] = title 

min = {}
min.update(minla)
f = open("min.json", "w")
f.write(json.dumps(min, indent=2))
f.close() 
# display json in textarea
widgeText("Minimum LA", minla,'150px' )

Textarea(value='{\n  "@context": "https://linked.art/ns/v1/linked-art.json",\n  "id": "https://abc.org/object/…

In [137]:
%%html

<div id="min" class="svg-small"></div>
<script>jsonvis("min.json", "#min")</script>

### Identifiers

https://linked.art/model/base/#identifiers

Many resources of interest are also given external identifiers, such as accession numbers for objects, ORCIDs for people or groups, lot numbers for auctions, and so forth. Identifiers are represented in a very similar way to names, but instead use the Identifier class. Identifiers will normally have a classification determining which sort of identifier it is, to distinguish between internal repository system assigned numbers from museum assigned accession numbers, for example.

As Identifiers and Names use the same `identified_by` property, the JSON will frequently have mixed classes in the array. Unlike Names, Identifiers are not part of human language and thus cannot have translations or a language associated with them.

In [112]:
jdoc = None
jdoc = {}

# define properties variables  
for prop in obj["atom"]:      
    propName = prop["@name"]        
    if "#text" in prop:
        if propName == "irn":
            irn = prop["#text"]    
        if propName == "TitAccessionNo":
            titAccessionNo = prop["#text"]
                               
jdoc["identified_by"] = []          
jdoc["identified_by"].append({
        "id": baseURI + "object/" + irn + "/irn",
        "type": "Identifier",
        "_label": "IMA at Newfields Collections Database Number for the Object",
        "content": irn,
        "classified_as": [{
            "id": "http://vocab.getty.edu/aat/300404621",
            "type": "Type",
            "_label": "repository numbers"
                        }]
                })
                   
jdoc["identified_by"].append({
        "id": baseURI + "/>object/" + irn + "/object-number",
        "type": "Identifier",
        "_label": "IMA at Newfields Object Number for the Object",
        "content": titAccessionNo,
        "classified_as": [{
            "id": "http://vocab.getty.edu/aat/300312355",
            "type": "Type",
            "_label": "accession numbers"
                        }]
                    })

identifiers = jdoc
frag = {}
frag.update(minla) 
frag.update(identifiers)

f = open("ids.json", "w")
f.write(json.dumps(frag, indent=2))
f.close() 

widgeText("Identifiers", frag,'200px' )

Textarea(value='{\n  "@context": "https://linked.art/ns/v1/linked-art.json",\n  "id": "https://abc.org/object/…

In [139]:
%%html
<script>jsonvis("ids.json", '#ids')</script>
<div id="ids" class="svg-small"/>

### Names

https://linked.art/model/base/#names

As the `_label` property is intended as internal documentation for the data, it is strongly recommended that every resource that should be rendered to an end user also have at least one specific name. The name could be for an object, a person, a group, an event or anything else. This pattern uses the `identified_by` property, with a `Name` resource. The value of the name is given in the content property of the `Name`.

It is somewhat unintuitive to think of a name as identifying the resource it is associated with, as names are typically not unique. However, as the name itself is uniquely identified rather than just an anonymous string, they are no longer a shared label and instead the particular instance of a name is uniquely associated with the resource. With this formulation, the name instance does uniquely identify the resource.

If there is more than one name given, then there should be one that is `classified_as` the primary name for use. This is done by adding the `Primary Name (aat:300404670) term to it. There should be exactly one primary title given per language.

Names are also part of human communication, and can have the Linguistic features of the model associated with them, such as having a particular language, or having translations.

In [114]:
jdoc = {}
jdoc["identified_by"] = []

# define properties variables  
for prop in obj["atom"]:      
    propName = prop["@name"] 
    if "#text" in prop:
        if propName == "irn":
            irn = prop["#text"]   
        if propName == "TitMainTitle":
            title = prop["#text"]
            

jdoc["identified_by"].append({
        "id": baseURI + "object/" + irn + "/title",
        "type": "Name",
        "_label": "Primary Title for the Object",
        "content": title ,
        "classified_as": [{
        "id": "http://vocab.getty.edu/aat/300404670",
        "type": "Type",
        "_label": "preferred terms"
                        }]
                })
try:   
    if obj["table"]["@name"] == "AltTitles":
        x = 0
        for tuple in obj["table"]["tuple"]:
            x +=1
            for atom in tuple:
                if atom["@name"] == "TitAlternateTitles":
                    content = atom["#text"]
                else:
                    content = ""
                    
                jdoc["identified_by"].append({
                    "id": baseURI + "object/" + irn + "/alt-title-" + x,
                    "type": "Name",
                    "_label": "Alternate Title for the Object",
                    "content": content,
                    "classified_as": [{
                            "id": "http://vocab.getty.edu/aat/300417227",
                            "type": "Type",
                            "_label": "alternate titles"}]   
                    })
except:
    pass
       
titles = jdoc   
frag = {}
frag.update(minla)
frag.update(titles)

f = open("name.json", "w")
f.write(json.dumps(frag, indent=2))
f.close() 

widgeText("Names", frag,'200px' )

Textarea(value='{\n  "@context": "https://linked.art/ns/v1/linked-art.json",\n  "id": "https://abc.org/object/…

In [140]:
%%html
<script>jsonvis("name.json", '#name')</script>
<div id="name" class="svg-small"/>

### Classification

https://linked.art/model/base/#types-and-classifications

CIDOC-CRM is a framework that must be extended via additional vocabularies and ontologies to be useful. The provided mechanism for doing this is the classified_as property, which refers to a term from a controlled vocabulary. This is in contrast to the `type` property, which is used for CIDOC-CRM defined classes, and a few extensions as needed. 

The `classified_as` property is thus a way to be more specific about the sort of entity, while maintaining the core information as the class using type. Controlled vocabulary entries should not be used with `type`, nor classes used with `classified_as`.

While any external vocabulary of terms can be used, the Getty's Art and Architecture Thesaurus is used whenever possible for consistency and that it is already widespread in the museum domain. The set of terms that have been identified as useful are listed in the community best-practices for recommendations, and within the documentation of the model when a particular choice is essential for interoperability.

In [62]:
jdoc = {}

# define properties variables  
for prop in obj["atom"]:      
    propName = prop["@name"] 
        
    if "#text" in prop:
        if propName == "TitObjectType":
            titleObjectType = prop["#text"]
        if propName == "irn":
            irn = prop["#text"]  
                           
phyMediaCategory = ""          
for table in obj["table"]:
    tableName = table["@name"]
    if tableName == "ObjectTypes":
        try:
            for atom in table["tuple"]:
                if atom["atom"]["@name"] == "PhyMediaCategory":
                    phyMediaCategory = atom["atom"]["#text"]
        except:
            pass
            
if titleObjectType.startswith("Visual Work"):
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300133025",
                    "type": "Type",
                    "_label": "works of art" 
                }]
                
if "Drawings" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300033973",
                    "type": "Type",
                    "_label": "drawings (visual works)" 
                }]
                
if "Multimedia" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300047910",
                    "type": "Type",
                    "_label": "multimedia works" 
                }]
                
if "Needlework" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300264072",
                    "type": "Type",
                    "_label": "needlework (visual works)" 
                }]
                
if "Paintings" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300033618",
                    "type": "Type",
                    "_label": "paintings (visual works)" 
                }] 
                
if "Pastel" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300076922",
                    "type": "Type",
                    "_label": "pastels (visual works)" 
                }] 
if "Performance" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300121445",
                    "type": "Type",
                    "_label": "performance art" 
                }] 
if "Photograph" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300046300",
                    "type": "Type",
                    "_label": "photographs" 
                }]
if "Prints" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300041273",
                    "type": "Type",
                    "_label": "prints (visual works)" 
                }]
if "Sculpture" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300047090",
                    "type": "Type",
                    "_label": "sculpture (visual works)" 
                }]
                
if phyMediaCategory != "":  
    try:
        atoms = table["tuple"]["atom"]
        for atom in atoms:
            if atom["@name"] == "PhyMediaCategory":
                    value = atom["#text"]
                    c = {
                    "id": baseURI + "thesauri/type/",
                    "type": "Type",
                    "_label": value
                    }
                    jdoc["classified_as"].append(c)
    except:
        pass
 
clasf = jdoc
frag = {}
frag.update(minla)
frag.update(clasf)

f = open("class.json", "w")
f.write(json.dumps(frag, indent=2))
f.close() 

widgeText("Classification", frag,'200px' )

Textarea(value='{\n  "@context": "https://linked.art/ns/v1/linked-art.json",\n  "id": "https://abc.org/object/…

In [141]:
%%html
<script>jsonvis("class.json", '#class')</script>
<div id="class" class="svg-small"/>

### Materials

https://linked.art/model/object/physical/#materials

Objects are created using different materials, such as canvas or marble. These are recorded using the `made_of` property on the object directly. The materials are the type of material, rather than the specific bits of matter and therefore refer to entries in external vocabularies. When possible, it is good to use this model, and combined with the parts model described in the next section, allows for a comprehensive set of information about which parts are which sizes, shapes, colors, and made of which materials.

Note that the type-of-type pattern is not needed for materials, like it is for shape, as they have their own `Material` class that is used to distinguish them.



In [64]:
jdoc = {}

made_of = []

# # <xsl:value-of select="lower-case(translate(replace(.,'[^a-zA-Z0-9 ]',''), ' ', '-'))"/>"
'''
for table in obj["table"]:
    if table["@name"] == "Medium":
        atom = table["tuple"].get("atom")    
        if atom.get("@name") == 'PhyMedium':
            value = atom.get("#text")
            made_of.append({
                    "id": baseURI + "thesauri/material/" + value,
                    "type": "Material",
                    "_label": "Material of Which the Object is Composed",
                    "content": value    
                    })
    if table["@name"] == "Support":
        atom = table["tuple"].get("atom")
        if atom.get("@name") == 'PhySupport':
            value = atom.get("#text")
            made_of.append({
                    "id": baseURI + "thesauri/material/" + value,
                    "type": "Material",
                    "_label": "Material of Which the Object is Composed",
                    "content": value    
                    })
'''
if len(made_of) > 0:
    jdoc["made_of"] = made_of

materials = {}
materials.update(jdoc)
    
frag = {}
frag.update(minla)
frag.update(materials)

widgeText("Materials", frag ,'200px' )

Textarea(value='{\n  "@context": "https://linked.art/ns/v1/linked-art.json",\n  "id": "https://abc.org/object/…

### Home Page
https://linked.art/model/digital/#home-page

A very common scenario is that there is a web page about the object, perhaps managed by a collections management system. For humans, this page is much more useful than the data intended for machines. It can be referenced with the `subject_of` property, and points to a `DigitalObject` which is `classified_as` a web page, or `aat:300264578`. As with digital images, the home page can have a format of "text/html" and other properties.


In [65]:
jdoc = None
jdoc = {}
homepageId = ""

# define properties variables  
for prop in obj["atom"]:      
    propName = prop["@name"]        
    if "#text" in prop:
        if propName == "irn":
            irn = prop["#text"]    
        if propName == "TitAccessionNo":
            titAccessionNo = prop["#text"]
       
for table in obj["table"]:
    
    tableName = table["@name"]
    if tableName == "Homepage":
        try:
            for atom in table["tuple"]["atom"]:
                if atom["@name"] == "EleIdentifier":
                    homepageId = atom["#text"]
        except:
            pass
                               
jdoc["subject_of"] = []  
jdoc["subject_of"].append({
        "id": "http://collection.imamuseum.org/artwork/" + homepageId,
        "type": "LinguisticObject",
        "_label": "Homepage for the Object",
        "classified_as": [
            {
            "id": "http://vocab/getty.edu/aat/300264578",
            "type": "Type",
            "_label": "Web pages (documents)"
            },
            {
            "id": "http://vocab.getty.edu/aat/300266277",
            "type": "Type",
            "_label": "home pages"
            }
        ],
        "format": "text/html"
                })
                  
homepage = jdoc
frag = {}
frag.update(minla) 
frag.update(homepage)

f = open("homepage.json", "w")
f.write(json.dumps(frag, indent=2))
f.close() 

from IPython.display import IFrame
IFrame("http://collection.imamuseum.org/artwork/" + homepageId, width='100%', height=800)

In [66]:
widgeText("Homepage", jdoc ,'200px' )

Textarea(value='{\n  "subject_of": [\n    {\n      "id": "http://collection.imamuseum.org/artwork/80085",\n   …

In [142]:
%%html
<script>jsonvis("homepage.json", '#home')</script>
<div id="home" class="svg-small"/>

### Current Location
https://linked.art/model/object/ownership/#location

- The current location of the object is given using the `current_location` property. 
- This can give a reference to a gallery or specific part of a facility, or be used for the general address of the organization where the object is currently held. 
- There are further modeling details available about [Places](https://linked.art/model/place/) on the Linked Art website.



In [68]:
jdoc = {}
jdoc["current_location"] = []

level2val = level3val = ""

for tuple in obj["tuple"]:
    
    if tuple['@name'] == 'LocCurrentLocationRef':
        for atom in tuple["atom"]:
            if atom["@name"] == 'LocLevel2':
                if atom["#text"] == 'see related parts':
                    see_related_parts = True
                else:
                    see_related_parts = False
                level2val = atom["#text"]
            if atom["@name"] == "LocLevel3":
                try:
                    level3val = atom["#text"]
                except:
                    level3val = ""
        
for tuple in obj["tuple"]:
    if tuple['@name'] == 'LocCurrentLocationRef':
        for atom in tuple["atom"]:
            #print(atom)
            if atom["@name"] == 'LocLevel1':    
                if atom["#text"] == 'On Loan':
                    jdoc["current_location"].append({
                        "id": baseURI + "thesauri/location/on-loan",
                        "type": "Place",
                        "_label": "On Loan"})
                    
                if 'Galler' in atom["#text"] or 'Suite' in atom["#text"]:
                    jdoc["current_location"].append({
                        "id": baseURI + "thesauri/location/" + level3val,
                        "type": "Place",
                        "_label": level2val,
                        "clasified_as": [{
                        "id": "http://vocab.getty.edu/aat/300240057",
                        "type": "Type",
                        "_label": "galleries (display spaces)"
                        }] })
            
            if atom["@name"] == 'LocLevel2': 
                if atom["#text"] == 'Efroymson Family Entrance':
                     jdoc["current_location"].append({ 
                         "id": baseURI + "thesauri/location/F02",
                        "type": "Place",
                        "_label": "Efroymson Family Entrance Pavilion"
                     })
                elif atom["#text"] == 'Nature Park':
                     jdoc["current_location"].append({ 
                         "id": baseURI + "thesauri/location/ANP",
                        "type": "Place",
                        "_label": "Virginia B. Fairbanks Art &amp; Nature Park"})
                        
                elif atom["#text"] == 'Grounds':
                    jdoc["current_location"].append({ 
                         "id": baseURI + "thesauri/location/G",
                        "type": "Place",
                        "_label": "Newfields Grounds"})
                    
                elif atom["#text"] == 'Asian Visible Storage':
                     jdoc["current_location"].append({ 
                         "id": baseURI + "thesauri/location/K241",
                        "type": "Place",
                        "_label": "Leah and Charles Reddish Gallery - Asian Visible Storage",
                         "classified_as": [
                            {
                            "id": "http://vocab.getty.edu/aat/300240057",
                            "type": "Type",
                            "_label": "galleries (display spaces)"
                            }]
                     })
                elif atom["#text"] == 'Westerley':
                    jdoc["current_location"].append({ 
                         "id": baseURI + "thesauri/location/westerley",
                        "type": "Place",
                        "_label": "Westerley"})    
                
                else:
                    jdoc["current_location"].append({ 
                         "id": baseURI + "thesauri/location/storage",
                        "type": "Place",
                        "_label": "IMA Storage"})  

location = jdoc
frag = {}
frag.update(minla) 
frag.update(location)

f = open("location.json", "w")
f.write(json.dumps(frag, indent=2))
f.close() 

widgeText("Location", jdoc ,'200px' )

Textarea(value='{\n  "current_location": [\n    {\n      "id": "https://abc.org/thesauri/location/storage",\n …

In [143]:
%%html
<script>jsonvis("location.json", '#loc')</script>
<div id="loc" class="svg-small"/>

### Statements about a Resource - Linguistic Objects 

https://linked.art/model/base/#statements-about-a-resource
    
In many cases, current data does not support the level of specificity that the full ontology allows, or the information is simply best expressed in human-readable form. For example, instead of a completely modeled set of parts with materials, many museum collection management systems allow only a single human-readable string for the "medium" or "materials statement". The same is true in many other situations, including rights or allowable usage statements, dimensions, edition statements and so forth. Any time that there is a description of the resource, with or without qualification as to the type of description, then this pattern can be used to record the descriptive text.

The pattern makes use of the `LinguisticObject` class that is used to identify a particular piece of textual content. These Linguistic Objects are then refered to by any other resource. They maintain the statement's text in the content property, and the language of the statement (if known) in the language property.

Use cases for this pattern include:

- General description of the resource
- Materials statement for an object
- Attribution statement for an image
- Biography for a person
- Dimensions statement for a part of an object    

In [71]:
jdoc = {}

jdoc["referred_to_by"] = []


for atom in obj["atom"]:
    if atom["@name"] == "SumCreditLine":
        val = atom["#text"]
        jdoc["referred_to_by"].append({
                 "id": baseURI + "object/" + irn + "/credit-line",
                "type": "LinguisticObject",
                "_label": "IMA at Newfields Credit Line for the Object",
                "content": val,
                "classified_as": [
                        {
                            "id": "http://vocab.getty.edu/aat/300026687",
                            "type": "Type",
                            "_label": "acknowledgments"
                        },
                        {
                            "id": "http://vocab.getty.edu/aat/300418049",
                            "type": "Type",
                            "_label": "brief texts"
                        }]
        })
    if atom["@name"] == 'CreProvenance':
        if "#text" in atom.keys():
            val = atom["#text"]
            jdoc["referred_to_by"].append({
                "id": baseURI + "object/" + irn + "/provenance-statement",
                    "type": "LinguisticObject",
                    "_label": "IMA Provenance Statement about the Object",
                    "content": val,
                    "classified_as": [
                        {
                            "id": "http://vocab.getty.edu/aat/300055863",
                            "type": "Type",
                            "_label": "provenance (history of ownership)"
                        },
                        {
                            "id": "http://vocab.getty.edu/aat/300418049",
                            "type": "Type",
                            "_label": "brief texts"
                        }
                    ]
            })
            
   


ling = jdoc
frag = {}
frag.update(minla) 
frag.update(ling)
   

f = open("linguistic.json", "w")
f.write(json.dumps(frag, indent=2))
f.close() 

widgeText("Statements", jdoc ,'200px' )

Textarea(value='{\n  "referred_to_by": [\n    {\n      "id": "https://abc.org/object/21137/credit-line",\n    …

In [144]:
%%html
<script>jsonvis("linguistic.json", '#ling')</script>
<div id="ling" class="svg-small"/>

## Compile Linked Art JSON-LD Representation

Bring the separate descriptions together in a single JSON-LD document

In [73]:

jdoc = {}

jdoc.update(minla)

if 'clasf' in vars():
    jdoc.update(clasf)
    
if 'homepage' in vars():
    jdoc.update(homepage)

if 'materials' in vars():
    jdoc.update(materials)
    
identified_by = identifiers.copy()
titles_items  = titles["identified_by"]
identified_by["identified_by"].append(titles_items[0])
jdoc.update(identified_by)

if 'location' in vars():
    jdoc.update(location)
    
if 'ling' in vars():
    jdoc.update(ling)

### View the final Linked Art JSON-LD representation for the selected object

The final Linked Art JSON-LD representation of the object is now available:

In [74]:
f = open("object_linkedart.json", "w")
f.write(json.dumps(jdoc, indent=2))
f.close() 

widgeText("Linked Art", jdoc ,'200px' )

Textarea(value='{\n  "@context": "https://linked.art/ns/v1/linked-art.json",\n  "id": "https://abc.org/object/…

### Visualise Linked Art JSON-LD as Graph

In [145]:
%%html
<script>jsonvis("object_linkedart.json", '#visualized')</script>
<div id="visualized" class="svg" ></div>

## View and Download the Object Linked Art JSON-LD File

The final JSON-LD file for the selected object is available at:
- [Selected Object as Linked Art](object_linkedart.json)

## Further Reading

Visit the [Linked Art website](https://linked.art) for further information on the data model and community activities